# Information Retrieval Assignment  
## Part 2B: KMeans Clustering

---

For the second improvement to our IR system, we decided to go with KMeans Clustering for the document vectors. This allows us to split up the documents into K groups with a centroid representing each group. We first compute the similarity of the query with the centroid and then compute the similarity of the query with the documents from the group whose centroid had the highest similarity. 

This allows us to speed up retreival time, since now we are only computing the similarity between N/K documents instead if N. However, to set up useful centroids, we must train the kmeans algorithm for more iterations, which adds to our setup time and overhead.

In [1]:
import string
import re
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

Here we load up the document vectors which were generated from the Part 1A code. This might take a minute or two.

In [2]:
try:
    document_vectors = pd.read_csv('vector_space_model.csv', index_col=0, keep_default_na=False)
except:
    print('File not found. Please run the Vector Space Model file first or move vector_space_model.csv to the same directory as this notebook')

In [3]:
document_vectors

,tsalta baptiste,karl richter (tennis),matrix ab,sergey golubitskiy,west side story (earl hines album),2016 tipperary senior hurling championship,mario mosböck,faith baptist college,volodymyr dykyi,félix baumaine,...,pinkwash (band),promo azteca,strange creek (west virginia),strange creek,collective sigh,dileep agrawal,strouds creek,mystic marathon,the daddy issues,vicky astori
,4,0,0,16,0,0,0,0,0,3,...,0,0,0,0,0,0,0,1,0,2
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
𐩦𐩧𐩢,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
𐩦𐩲𐩧𐩣,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
𐩱𐩡,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
𐩱𐩥𐩩𐩧𐩣,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Extracting the document list and vocabulary dictionary from the pandas dataframe.

In [4]:
docs = document_vectors.columns
vocab_dict = {term: termID for termID, term in enumerate(document_vectors.index)}

Length normalisation for documents so that clusters are dependant more on context similarity rather than document length.

In [5]:
kmeans_input = np.divide(document_vectors, np.linalg.norm(document_vectors, axis=0), dtype=np.float)

### KMeans Clustering Algorithm

We have chosen $k = \sqrt{N}$ where $N$ is the total number of documents.

`n_init` is the number of times the kmeans algorithm will be run with different starting centroids. The final output will be the best performing run.   
`max_iter` is the number of iterations, where it will update the centroids.

We have not been able to fine tune these parametes as run takes around half an hour to complete, even after lowering `n_init` and `max_iter`.

In [6]:
%%time
K = int(np.sqrt(len(docs)))
kmeans = KMeans(n_clusters=K, n_init=5, max_iter=50, random_state=42, verbose=1).fit(kmeans_input.T)

Initialization complete
Iteration 0, inertia 4556.161205259165
Iteration 1, inertia 2946.6899070757836
Iteration 2, inertia 2892.4140513056564
Iteration 3, inertia 2873.616141174195
Iteration 4, inertia 2863.0056758481965
Iteration 5, inertia 2855.3665706198944
Iteration 6, inertia 2852.712331650997
Iteration 7, inertia 2851.446216736263
Iteration 8, inertia 2850.0938925410337
Iteration 9, inertia 2848.9323978091898
Iteration 10, inertia 2848.303350962511
Iteration 11, inertia 2847.7693484535107
Iteration 12, inertia 2847.3556882091866
Iteration 13, inertia 2846.96834393751
Iteration 14, inertia 2846.407809408285
Iteration 15, inertia 2846.1206792723806
Iteration 16, inertia 2845.8928847150537
Iteration 17, inertia 2845.7045592651375
Iteration 18, inertia 2845.427162102591
Iteration 19, inertia 2844.9229884359675
Iteration 20, inertia 2844.6504763393104
Iteration 21, inertia 2844.3525730385054
Iteration 22, inertia 2844.164975882181
Iteration 23, inertia 2843.9684782249647
Iteration 24

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_fast._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt: 


Iteration 1, inertia 2920.2472947229858
Iteration 2, inertia 2862.488799018512
Iteration 3, inertia 2841.8958792162166
Iteration 4, inertia 2830.405994149125
Iteration 5, inertia 2823.4214338276697
Iteration 6, inertia 2819.1458054597438
Iteration 7, inertia 2816.2011530133286
Iteration 8, inertia 2814.052692924209
Iteration 9, inertia 2812.078868495841
Iteration 10, inertia 2810.3596107028225
Iteration 11, inertia 2809.2471379118665
Iteration 12, inertia 2808.224164670245
Iteration 13, inertia 2807.8055410981733
Iteration 14, inertia 2807.561123812709
Iteration 15, inertia 2807.4969571863235
Iteration 16, inertia 2807.471997784187
Iteration 17, inertia 2807.461813797582
Iteration 18, inertia 2807.4481192458043
Iteration 19, inertia 2807.435515971924
Iteration 20, inertia 2807.42438435603
Iteration 21, inertia 2807.4195974158174
Iteration 22, inertia 2807.408809073786
Iteration 23, inertia 2807.3902535170514
Iteration 24, inertia 2807.3839554340834
Converged at iteration 24: strict con

### Computing the lnc for centroids:

$$ Logarithm\ "term\ frequency" \left( l \right): 1 + \log_{2}\left ( tf_{t,c} \right ) $$  

$$ No\ "document\ frequency" \left( n \right): 1 $$  

$$ Cosine\ normalisation \left( c \right): \sqrt{ \sum\limits_{i=1}^{T} {w^{2}_{i}}} $$  

Having the same scoring scheme between query and clusters and query and documents makes sure that if a query has high similarity with the centroid, it will have high similarity with the cluster.

In [7]:
#(l)nc for centroids
w_c = kmeans.cluster_centers_.T
w_c = np.array(w_c, dtype=np.float64)
w_c = np.log2(w_c, out=np.zeros_like(w_c), where=(w_c>=1.0))
w_c = np.add(w_c, 1, out=np.zeros_like(w_c), where=(w_c!=0))

#ln(c) for documents

w_c = np.divide(w_c, np.linalg.norm(w_c, axis=1).reshape(-1,1))
w_c[np.isnan(w_c)] = 0

<ipython-input-7-295bcc239888>:9: RuntimeWarning: invalid value encountered in true_divide
  w_c = np.divide(w_c, np.linalg.norm(w_c, axis=1).reshape(-1,1))


---

## Test Queries

We have provided some sample queries to try out. Please uncomment the relevant line before running the query. There is also an option to enter your own query.

In [8]:
# query = "football footballer midfielder"
# query = "north american beaver"
# query = "car ride"
# query = "falling off the edge of the world"
# query = "tax planning strategies"
# query = "guitar musician"
# query = "jordan peterson"

query = input()

query_vector = np.array([0]*len(vocab_dict), dtype=np.float)
for word in query.strip().split(' '):
    if word in vocab_dict:
        query_vector[vocab_dict[word]] += 1

football footballer midfielder


### Computing the ltc for queries:

$$ Logarithm\ term\ frequency \left( l \right): 1 + \log_{2}\left ( tf_{t,d} \right ) $$ 

$$ Inverse\ document\ frequency \left( n \right): \log_{2}\left(\frac{N+1}{df_{t}} \right) $$   

$$ Cosine\ normalisation \left( c \right): \sqrt{ \sum\limits_{i=1}^{T} {w^{2}_{i}}} $$  


In [9]:
#(l)tc for query vector

w_q = query_vector
w_q = np.log2(w_q, out=np.zeros_like(w_q), where=(w_q!=0))
w_q = np.add(w_q, 1)

#l(t)c for query vector

idf = np.array(document_vectors, dtype=np.float64)
idf = np.sum(np.array(idf!=0), axis=1, dtype=np.float64)
idf = np.log2((len(docs)+1)/idf, out=np.zeros_like(idf), where=(idf!=0))

#lt(c) for query vector

w_q = np.multiply(w_q, idf)
w_q = np.divide(w_q, np.linalg.norm(w_q))

### Calculating Cosine Similarity between query and centroids:

$$ \cos \left( d_{j}, q \right) = \frac {\sum\limits_{i=1}^{N}{w_{i,j}w_{i,q}}}{\sqrt{\sum\limits_{i=i}^{N}w_{i,j}^{2}} \sqrt{\sum\limits_{i=i}^{N}w_{i,q}^{2}}} $$     

Cosine similarity lies in the range [0,1], with 0 meaning no similarity and 1 meaning complete similarity. 

Since, we have already normalised the vectors, we just need to get their dot product to compute the cosine similarity. To optimise performance, we are computing the scores by matrix multiplication instead of running a loop.

In [10]:
scores = np.multiply(w_q.reshape(-1,1), w_c)
scores = np.sum(scores, axis=0)
scores[np.isnan(scores)] = 0

These are the top 10 centroids and their scores. The top centroid is chosen and the cosine similarity is again calculated between the query and the cluster belonging to the top centroid.

In [11]:
for i in np.argsort(scores)[-10:][::-1]:
  print("%.8f" % scores[i], i)

0.00000000 75
0.00000000 27
0.00000000 20
0.00000000 21
0.00000000 22
0.00000000 23
0.00000000 24
0.00000000 25
0.00000000 26
0.00000000 28


In [12]:
top_centroid = np.argsort(scores)[-1]
ls = kmeans.labels_ == top_centroid
print(ls.sum())
top_cluster = document_vectors.loc[:,ls]
top_cluster

112


,matrix ab,crack international art camp,t. p. ramakrishnan,madrid–barcelona railway,devinyl splits no. 5,marea,daphne kosaninii,daphne jezoensis,philippe moggio,pirivom santhippom (season 2),...,medina del campo railway station,palencia railway station,julius caesar (block wargame),border road,gettysburg (block wargame),león railway station,yuka takeshima,iwi dan .338 sniper rifle,spencer creek,tranquility (lee konitz album)
,0,10,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
𐩦𐩧𐩢,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
𐩦𐩲𐩧𐩣,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
𐩱𐩡,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
𐩱𐩥𐩩𐩧𐩣,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Computing the lnc for documents:

$$ Logarithm\ term\ frequency \left( l \right): 1 + \log_{2}\left ( tf_{t,c} \right ) $$  

$$ No\ document\ frequency \left( n \right): 1 $$  

$$ Cosine\ normalisation \left( c \right): \sqrt{ \sum\limits_{i=1}^{T} {w^{2}_{i}}} $$  

Here, the score is being calculated for only the $\frac{N}{k}$ documents belonging to the top cluster.

In [14]:
#(l)nc for documents in top cluster
w_d = top_cluster
w_d = np.array(w_d, dtype=np.float64)
w_d = np.log2(w_d, out=np.zeros_like(w_d), where=(w_d>=1.0))
w_d = np.add(w_d, 1, out=np.zeros_like(w_d), where=(w_d!=0))

#ln(c) for documents

w_d = np.divide(w_d, np.linalg.norm(w_d, axis=1).reshape(-1,1))
w_d[np.isnan(w_d)] = 0

<ipython-input-14-21f9917c235b>:9: RuntimeWarning: invalid value encountered in true_divide
  w_d = np.divide(w_d, np.linalg.norm(w_d, axis=1).reshape(-1,1))


### Calculating Cosine Similarity between query and centroids:

$$ \cos \left( d_{j}, q \right) = \frac {\sum\limits_{i=1}^{N}{w_{i,j}w_{i,q}}}{\sqrt{\sum\limits_{i=i}^{N}w_{i,j}^{2}} \sqrt{\sum\limits_{i=i}^{N}w_{i,q}^{2}}} $$ 

Cosine similarity lies in the range [0,1], with 0 meaning no similarity and 1 meaning complete similarity. 

Since, we have already normalised the vectors, we just need to get their dot product to compute the cosine similarity. To optimise performance, we are computing the scores by matrix multiplication instead of running a loop.

In [15]:
scores = np.multiply(w_q.reshape(-1,1), w_d)
scores = np.sum(scores, axis=0)
scores[np.isnan(scores)] = 0

---

## Result

Here are the top 10 documents and their scores calculated by the IR system optimised with KMeans clustering.

In [16]:
rank = np.argsort(scores)[-10:][::-1]

print("   score  | name of document")
print("-------------------------------")
for i in rank:
    print("%.8f" % scores[i], docs[i])

   score  | name of document
-------------------------------
0.37211641 karl richter (tennis)
0.07718727 william strahan (cricketer)
0.07313320 henrich danckwardt
0.05580404 șuica river
0.05347079 lee seo-won
0.05155923 olga fedorovich
0.05074536 2008–12 cyprus talks
0.04727641 daniel karaba
0.04632542 manuel martic
0.04554281 nick robertson (businessman)


## Observations

- While it does work, the results are far from great. To get better results, we will have to increase the value of $K$, which is not possible right now as the system is already running at its peak memory.    

- Also, there is a huge deviation in the number of documents per cluster. This might be due to the fact that length normalisation is not having its desired effect or due to presence of non-english terms in the vocabulary which are clustering together.